In [15]:
import numpy as np
from sympy import *

from MassMatrix.wrapper_module_0 import autofunc_c as MassMatrix
from CoriolisVector.wrapper_module_1 import autofunc_c as CoriolisVector
from Ls.wrapper_module_2 import autofunc_c as Ls
from Lm.wrapper_module_3 import autofunc_c as Lm
from Ls_derivative.wrapper_module_4 import autofunc_c as Ls_d
from Lm_derivative.wrapper_module_5 import autofunc_c as Lm_d

from pympc.geometry.polyhedron import Polyhedron
from pympc.dynamics.discrete_time_systems import LinearSystem
from pympc.control.controllers import ModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_state_space_trajectory


In [16]:
def mass_matrix(spacecraft_angles=None, joint_angles=None, Is=None, I_link=None, mass=None):
    if not Is:
        Is = np.array([1400, 1400, 2040])
    if not I_link:
        I_link = np.array([0.10, 0.25, 0.25, 0.10, 0.26, 0.26, 0.10, 0.26, 0.26])  # Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3
    if not mass:
        mass = np.array([200.0, 20.0, 50.0, 50.0]) # m0=mass of satellite and the rest are link masses
    Is_xx, Is_yy, Is_zz = Is
    Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3 = I_link
    alpha, beta, gamma = spacecraft_angles
    m0, m1, m2, m3 = mass
    theta_1, theta_2, theta_3 = joint_angles
    return MassMatrix(Is_xx, Is_yy, Is_zz, Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3, alpha, beta, gamma, m0, m1, m2, m3, theta_1, theta_2, theta_3)

In [17]:
def coriolis_vector(spacecraft_angles=None, joint_angles=None, spacecraft_vel=None, joint_vel=None, Is=None, I_link=None, mass=None):
    if not Is:
        Is = np.array([1400, 1400, 2040])
    if not I_link:
        I_link = np.array([0.10, 0.25, 0.25, 0.10, 0.26, 0.26, 0.10, 0.26, 0.26])  # Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3
    if not mass:
        mass = np.array([200.0, 20.0, 50.0, 50.0]) # m0=mass of satellite and the rest are link masses
    Is_xx, Is_yy, Is_zz = Is
    Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3 = I_link
    alpha, beta, gamma = spacecraft_angles
    m0, m1, m2, m3 = mass
    theta_1, theta_2, theta_3 = joint_angles
    alpha_d, beta_d, gamma_d = spacecraft_vel
    theta_1d, theta_2d, theta_3d = joint_vel
    return CoriolisVector(Is_xx, Is_yy, Is_zz, Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3, alpha, alpha_d, beta, beta_d, gamma, gamma_d, m0, m1, m2, m3, theta_1, theta_1d, theta_2, theta_2d, theta_3, theta_3d)

In [ ]:
def calculate_Ls(spacecraft_angles=None, joint_angles=None, Is=None, I_link=None, mass=None):
    if not Is:
        Is = np.array([1400, 1400, 2040])
    if not I_link:
        I_link = np.array([0.10, 0.25, 0.25, 0.10, 0.26, 0.26, 0.10, 0.26, 0.26])  # Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3
    if not mass:
        mass = np.array([200.0, 20.0, 50.0, 50.0]) # m0=mass of satellite and the rest are link masses
    Is_xx, Is_yy, Is_zz = Is
    Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3 = I_link
    alpha, beta, gamma = spacecraft_angles
    m0, m1, m2, m3 = mass
    theta_1, theta_2, theta_3 = joint_angles
    return Ls(Is_xx, Is_yy, Is_zz, Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3, alpha, beta, gamma, m0, m1, m2, m3, theta_1, theta_2, theta_3)

In [ ]:
def calculate_Lm(spacecraft_angles=None, joint_angles=None, Is=None, I_link=None, mass=None):
    if not Is:
        Is = np.array([1400, 1400, 2040])
    if not I_link:
        I_link = np.array([0.10, 0.25, 0.25, 0.10, 0.26, 0.26, 0.10, 0.26, 0.26])  # Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3
    if not mass:
        mass = np.array([200.0, 20.0, 50.0, 50.0]) # m0=mass of satellite and the rest are link masses
    Is_xx, Is_yy, Is_zz = Is
    Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3 = I_link
    alpha, beta, gamma = spacecraft_angles
    m0, m1, m2, m3 = mass
    theta_1, theta_2, theta_3 = joint_angles
    return Lm(Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3, alpha, beta, gamma, m0, m1, m2, m3, theta_1, theta_2, theta_3)

In [ ]:
def Ls_derivative(spacecraft_angles=None, joint_angles=None, spacecraft_vel=None, joint_vel=None, Is=None, I_link=None, mass=None):
    if not Is:
        Is = np.array([1400, 1400, 2040])
    if not I_link:
        I_link = np.array([0.10, 0.25, 0.25, 0.10, 0.26, 0.26, 0.10, 0.26, 0.26])  # Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3
    if not mass:
        mass = np.array([200.0, 20.0, 50.0, 50.0]) # m0=mass of satellite and the rest are link masses
    Is_xx, Is_yy, Is_zz = Is
    Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3 = I_link
    alpha, beta, gamma = spacecraft_angles
    m0, m1, m2, m3 = mass
    theta_1, theta_2, theta_3 = joint_angles
    alpha_d, beta_d, gamma_d = spacecraft_vel
    theta_1d, theta_2d, theta_3d = joint_vel
    return Ls_d(Is_xx, Is_yy, Is_zz, Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3, alpha, alpha_d, beta, beta_d, gamma, gamma_d, m0, m1, m2, m3, theta_1, theta_1d, theta_2, theta_2d, theta_3, theta_3d)

In [ ]:
def Lm_derivative(spacecraft_angles=None, joint_angles=None, spacecraft_vel=None, joint_vel=None, Is=None, I_link=None, mass=None):
#     if not Is:
#         Is = np.array([1400, 1400, 2040])
    if not I_link:
        I_link = np.array([0.10, 0.25, 0.25, 0.10, 0.26, 0.26, 0.10, 0.26, 0.26])  # Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3
    if not mass:
        mass = np.array([200.0, 20.0, 50.0, 50.0]) # m0=mass of satellite and the rest are link masses
#     Is_xx, Is_yy, Is_zz = Is
    Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3 = I_link
    alpha, beta, gamma = spacecraft_angles
    m0, m1, m2, m3 = mass
    theta_1, theta_2, theta_3 = joint_angles
    alpha_d, beta_d, gamma_d = spacecraft_vel
    theta_1d, theta_2d, theta_3d = joint_vel
    return Lm_d(Ixx1, Ixx2, Ixx3, Iyy1, Iyy2, Iyy3, Izz1, Izz2, Izz3, alpha, alpha_d, beta, beta_d, gamma, gamma_d, m0, m1, m2, m3, theta_1, theta_1d, theta_2, theta_2d, theta_3, theta_3d)

In [18]:
phi_s = np.array([0.1, 0.2, 0.3])  # alpha, beta, gamma
q = np.array([0.01, 0.2, 0.15])  # theta_1, theta_2, theta_3
phi_s_dot = np.array([.1, .2, .3])
q_dot = np.array([.3, .4, .5])

In [32]:
# mass_matrix(phi_s, q)
# coriolis_vector(phi_s, q, phi_s_dot, q_dot)
# calculate_Ls(phi_s, q)
# calculate_Lm(phi_s, q)
# Ls_derivative(phi_s, q, phi_s_dot, q_dot)
# Lm_derivative(phi_s, q, phi_s_dot, q_dot)